Convert GA output into something usable:

In [1]:
import h5py
import numpy as np 
import netCDF4 as nc
import os 

n_parameters = 12
candidates_per_iteration = 30
n_radial_points = 150
path = "../data/genetic_algorithm"

generation = 0
with h5py.File(f'{path}/output.hdf5', 'w') as hdf5:
    with open(f'{path}/genetic-algorithm_out.txt') as f:
        for line in f.readlines():
            l = line.strip()
            if l.startswith('GENERATION'):
                if generation > 0:
                    # Save previous generation
                    hdf5[f"{generation}/parameters"] = parameters
                    hdf5[f"{generation}/fitness"] = fitness
                    hdf5[f"{generation}/converged_ecrh"] = converged_ecrh
                    hdf5[f"{generation}/converged_q"] = converged_q
                # Initialise storage for next generation
                generation += 1
                parameters = np.empty((candidates_per_iteration, n_parameters))
                fitness = np.empty(candidates_per_iteration)
                converged_ecrh = np.zeros((candidates_per_iteration, n_radial_points))
                converged_q = np.zeros((candidates_per_iteration, n_radial_points))
                candidate = 0
            if l.startswith('Bred') or l.startswith('Culled'):
                cols = [c.strip() for c in l.split('|')]
                dirname = cols[3].strip()
                profiles_path = f"{path}/{dirname}/profiles.CDF"
                if os.path.exists(profiles_path):
                    profiles = nc.Dataset(profiles_path)
                    if "QECE" in profiles.variables and "Q" in profiles.variables:
                        converged_ecrh[candidate] = profiles["QECE"][-1]
                        converged_q[candidate] = profiles["Q"][-1]
                parameters[candidate] = [float(p) for p in dirname.split('_')[-n_parameters:]]
                fitness[candidate] = float(cols[1])
                candidate += 1

In [2]:
from typing import Iterable

def parameters_to_dirname(parameters: Iterable[float]):
    return "../data/genetic_algorithm/Chromosome_spr045_v9d0__" + "_".join([f"{p:.2f}" for p in parameters])

Check that our ECRH parameterisations match:

In [3]:
from jetto_mobo.genetic_algorithm import piecewise_linear
import plotly.graph_objects as go
import h5py
import numpy as np 
from plotly.subplots import make_subplots

with h5py.File("../data/genetic_algorithm/output.hdf5") as hdf5:
    converged_ecrh = hdf5["1/converged_ecrh"][0]
    x = np.linspace(0, 1, len(converged_ecrh))
    parameters = hdf5["1/parameters"][0]
    target_ecrh = piecewise_linear(x, parameters)
    

figure = make_subplots(specs=[[{"secondary_y": True}]])
figure.add_trace(
    go.Scatter(
        x=x,
        y=converged_ecrh,
        name="Normalised converged ECRH",
    )
)
figure.add_trace(
    go.Scatter(
        x=x,
        y=target_ecrh,
        name="Parameterisation"
    ),
    secondary_y=True
)
figure.show()

Check that our cost functions match:

In [4]:
from jetto_mobo.genetic_algorithm import scalar_objective
import h5py
import netCDF4

with h5py.File("../data/genetic_algorithm/output.hdf5") as hdf5:
    converged_q = hdf5["1/converged_q"][0]
    fitness = hdf5["1/fitness"][0]
    dirname = parameters_to_dirname(hdf5["1/parameters"][0])
    profiles = netCDF4.Dataset(f"{dirname}/profiles.CDF")
    timetraces = netCDF4.Dataset(f"{dirname}/timetraces.CDF")
    value = scalar_objective(profiles, timetraces)
    
print(fitness, value)

0.89188 0.8877162096497472
